<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Functions-for-web-scripting" data-toc-modified-id="Functions-for-web-scripting-1"><span class="toc-item-num">1&nbsp;&nbsp;</span><strong><em>Functions for web scripting</em></strong></a></span><ul class="toc-item"><li><span><a href="#Installation-of-the-necessary-libraries" data-toc-modified-id="Installation-of-the-necessary-libraries-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span><em>Installation of the necessary libraries</em></a></span></li><li><span><a href="#Importing-important-libraries" data-toc-modified-id="Importing-important-libraries-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span><em>Importing important libraries</em></a></span></li><li><span><a href="#Functions-to-scroll-the-page" data-toc-modified-id="Functions-to-scroll-the-page-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span><em>Functions to scroll the page</em></a></span></li><li><span><a href="#Function-to-download-data-of-interest-from-the-following-sites:." data-toc-modified-id="Function-to-download-data-of-interest-from-the-following-sites:.-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span><em>Function to download data of interest from the following sites:</em>.</a></span></li><li><span><a href="#Function-that-converts-data-into-dictionaries-with-lists" data-toc-modified-id="Function-that-converts-data-into-dictionaries-with-lists-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span><em>Function that converts data into dictionaries with lists</em></a></span></li><li><span><a href="#Function-for-web-scripting." data-toc-modified-id="Function-for-web-scripting.-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span><em>Function for web scripting</em>.</a></span></li><li><span><a href="#Function-that-checks-the-number-of-pages-with-advertisements" data-toc-modified-id="Function-that-checks-the-number-of-pages-with-advertisements-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span><em>Function that checks the number of pages with advertisements</em></a></span></li></ul></li><li><span><a href="#Capture-all-data" data-toc-modified-id="Capture-all-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span><strong><em>Capture all data</em></strong></a></span><ul class="toc-item"><li><span><a href="#Adres-url" data-toc-modified-id="Adres-url-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span><em>Adres url</em></a></span></li><li><span><a href="#Data-extracted-from-web-scripting" data-toc-modified-id="Data-extracted-from-web-scripting-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span><em>Data extracted from web scripting</em></a></span></li></ul></li><li><span><a href="#Data-as-DataFrame" data-toc-modified-id="Data-as-DataFrame-3"><span class="toc-item-num">3&nbsp;&nbsp;</span><strong><em>Data as DataFrame</em></strong></a></span><ul class="toc-item"><li><span><a href="#Pandas-library-import" data-toc-modified-id="Pandas-library-import-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Pandas library import</a></span></li><li><span><a href="#Changing-the-object-to-a-DataFrame" data-toc-modified-id="Changing-the-object-to-a-DataFrame-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Changing the object to a DataFrame</a></span></li><li><span><a href="#Saving-the-object-as-a-csv-file" data-toc-modified-id="Saving-the-object-as-a-csv-file-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Saving the object as a csv file</a></span></li></ul></li></ul></div>

# ***House prices in Poland***

## ***Functions for web scripting***

### *Installation of the necessary libraries*

In [1]:
%pip install selenium 
%pip install beautifulsoup4
%pip install price_parser

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### *Importing important libraries*

In [2]:
import time 
import selenium
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from price_parser import Price

### *Functions to scroll the page*

In [3]:
def scrollDown(driver, value):
    driver.execute_script("window.scrollBy(0,"+str(value)+")")

In [4]:
def scrollDownAllTheWay(driver):
    old_page = driver.page_source
    while True:
#         print("Scrolling down...")
        for i in range(2):
            scrollDown(driver, 5000)
            time.sleep(0.2)
        new_page = driver.page_source
        if new_page != old_page:
            old_page = new_page
        else:
            break
    return True

### *Function to download data of interest from the following sites:*.
* the place where the house is located
* price per property
* price per m2
* number of rooms 
* area of the house 

In [5]:
def get_data_from_pages(html):
    info = []
    soup = BeautifulSoup(html, 'html.parser')
    all_search = soup.find('div', {'data-cy': "search.listing.organic"})
    voivodeship = soup.find('h1', class_='css-142cvbz elbi2ci1').text.split(':')[1].strip()
    
    for li in all_search.find_all('li', {"data-cy": "listing-item"}):
        city = li.find('p', class_='css-14aokuk e1ualqfi4').text
        div = li.find('div', class_="eel6co20 css-1ucvgmr ei6hyam0")
        all_span = div.find_all('span')
        date = [voivodeship, city]
        
        for span in all_span:
            date.append(span.text)
        info.append(date)
    
    new_info = [i for i in info if len(i) > 5 and i[2] != 'Zapytaj o cenę']
    
    for item in new_info:
        item[:] = [x.replace('\xa0', '') for x in item]
        item[2] = int(Price.fromstring(item[2]).amount)
        item[3] = Price.fromstring(item[3]).amount
        item[4] = int(item[4].replace(' pokój', '').replace(' pokoje', '').replace('pokoi', '').replace('+ ', ''))
        item[5] = int(float(item[5].replace(' m²', '')))
    
    return new_info

### *Function that converts data into dictionaries with lists*

In [6]:
def all_data_dict(info):
    data = {'voivodeship': [],'town': [], 'price': [], 'price per m2': [], 'number of rooms': [], 'area': []}
    seen_combinations = set()

    for item in info:
        combination = (item[0], item[1], item[2], item[3], item[4], item[5])

        if combination not in seen_combinations and item[5] <= 1000:
            seen_combinations.add(combination)
            data['voivodeship'].append(item[0])
            data['town'].append(item[1])
            data['price'].append(item[2])
            data['price per m2'].append(item[3])
            data['number of rooms'].append(item[4])
            data['area'].append(item[5])

    return data

### *Function for web scripting*.
A function that pulls data from a page and changes it 

In [7]:
def web_scripping(urls_pages):
    info = []
    driver = webdriver.Safari()
    count = 1
    print('Start')
    for url, number_of_pages in urls_pages.items():
        driver.get(url)
        print('Start voivodeship')
        start_time = time.time()
        for i in range(number_of_pages):
            if i == math.ceil(number_of_pages/2):
                print('50%')
            scrollDownAllTheWay(driver)
            html = driver.page_source
            info += get_data_from_pages(html)
            while True:
                try:
                    driver.execute_script("document.querySelector(\"button[data-cy='pagination.next-page']\").click()")
                    break
                except:
                    pass
            time.sleep(2)
        print('End of voivodeship: ', count)
        count += 1
        time.sleep(120)
    print('End webscripping!')
    driver.close()
    data = all_data_dict(info)
    return data

### *Function that checks the number of pages with advertisements*

In [8]:
from urllib.request import urlopen
import math
def urls_pages(urls):
    urls_numbers = {}
    for url in urls:
        page = urlopen(url)
        html = page.read().decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')
        number_of_offers = int(soup.find('span', class_='css-19fwpg e17mqyjp2').text)
        urls_numbers[url] = math.ceil(number_of_offers/72)
    return urls_numbers

## ***Capture all data***

### *Adres url*

In [9]:
urls = ['https://www.otodom.pl/pl/oferty/sprzedaz/mieszkanie/dolnoslaskie?distanceRadius=0&page=1&limit=72&ownerTypeSingleSelect=ALL&locations=%5Bregions-1%5D&by=DEFAULT&direction=DESC&viewType=listing'
        ,'https://www.otodom.pl/pl/oferty/sprzedaz/mieszkanie/kujawsko-pomorskie?distanceRadius=0&page=1&limit=72&ownerTypeSingleSelect=ALL&locations=%5Bregions-2%5D&by=DEFAULT&direction=DESC&viewType=listing'
        ,'https://www.otodom.pl/pl/oferty/sprzedaz/mieszkanie/lodzkie?distanceRadius=0&page=1&limit=72&ownerTypeSingleSelect=ALL&locations=%5Bregions-5%5D&by=DEFAULT&direction=DESC&viewType=listing'
        ,'https://www.otodom.pl/pl/oferty/sprzedaz/mieszkanie/lubelskie?distanceRadius=0&page=1&limit=72&ownerTypeSingleSelect=ALL&locations=%5Bregions-3%5D&by=DEFAULT&direction=DESC&viewType=listing'
        ,'https://www.otodom.pl/pl/oferty/sprzedaz/mieszkanie/lubuskie?distanceRadius=0&page=1&limit=72&ownerTypeSingleSelect=ALL&locations=%5Bregions-4%5D&by=DEFAULT&direction=DESC&viewType=listing'
        ,'https://www.otodom.pl/pl/oferty/sprzedaz/mieszkanie/malopolskie?distanceRadius=0&page=1&limit=72&ownerTypeSingleSelect=ALL&locations=%5Bregions-6%5D&by=DEFAULT&direction=DESC&viewType=listing'
        ,'https://www.otodom.pl/pl/oferty/sprzedaz/mieszkanie/mazowieckie?distanceRadius=0&page=1&limit=72&ownerTypeSingleSelect=ALL&locations=%5Bregions-7%5D&by=DEFAULT&direction=DESC&viewType=listing'
        ,'https://www.otodom.pl/pl/oferty/sprzedaz/mieszkanie/opolskie?distanceRadius=0&page=1&limit=72&ownerTypeSingleSelect=ALL&locations=%5Bregions-8%5D&by=DEFAULT&direction=DESC&viewType=listing'
        ,'https://www.otodom.pl/pl/oferty/sprzedaz/mieszkanie/podkarpackie?distanceRadius=0&page=1&limit=72&ownerTypeSingleSelect=ALL&locations=%5Bregions-9%5D&by=DEFAULT&direction=DESC&viewType=listing'
        ,'https://www.otodom.pl/pl/oferty/sprzedaz/mieszkanie/podlaskie?distanceRadius=0&page=1&limit=72&ownerTypeSingleSelect=ALL&locations=%5Bregions-10%5D&by=DEFAULT&direction=DESC&viewType=listing'
        ,'https://www.otodom.pl/pl/oferty/sprzedaz/mieszkanie/pomorskie?distanceRadius=0&page=1&limit=72&ownerTypeSingleSelect=ALL&locations=%5Bregions-11%5D&by=DEFAULT&direction=DESC&viewType=listing'
        ,'https://www.otodom.pl/pl/oferty/sprzedaz/mieszkanie/slaskie?distanceRadius=0&page=1&limit=72&ownerTypeSingleSelect=ALL&locations=%5Bregions-12%5D&by=DEFAULT&direction=DESC&viewType=listing'
        ,'https://www.otodom.pl/pl/oferty/sprzedaz/mieszkanie/swietokrzyskie?distanceRadius=0&page=1&limit=72&ownerTypeSingleSelect=ALL&locations=%5Bregions-13%5D&by=DEFAULT&direction=DESC&viewType=listing'
        ,'https://www.otodom.pl/pl/oferty/sprzedaz/mieszkanie/warminsko-mazurskie?distanceRadius=0&page=1&limit=72&ownerTypeSingleSelect=ALL&locations=%5Bregions-14%5D&by=DEFAULT&direction=DESC&viewType=listing'
        ,'https://www.otodom.pl/pl/oferty/sprzedaz/mieszkanie/wielkopolskie?distanceRadius=0&page=1&limit=72&ownerTypeSingleSelect=ALL&locations=%5Bregions-15%5D&by=DEFAULT&direction=DESC&viewType=listing'
        ,'https://www.otodom.pl/pl/oferty/sprzedaz/mieszkanie/zachodniopomorskie?distanceRadius=0&page=1&limit=72&ownerTypeSingleSelect=ALL&locations=%5Bregions-16%5D&by=DEFAULT&direction=DESC&viewType=listing']

### *Data extracted from web scripting*

In [10]:
data = web_scripping( urls_pages(urls))

Start
Start voivodeship
50%
End of voivodeship:  1
Start voivodeship
50%
End of voivodeship:  2
Start voivodeship
50%
End of voivodeship:  3
Start voivodeship
50%
End of voivodeship:  4
Start voivodeship
50%
End of voivodeship:  5
Start voivodeship
50%
End of voivodeship:  6
Start voivodeship
50%
End of voivodeship:  7
Start voivodeship
50%
End of voivodeship:  8
Start voivodeship
50%
End of voivodeship:  9
Start voivodeship
50%
End of voivodeship:  10
Start voivodeship
50%
End of voivodeship:  11
Start voivodeship
50%
End of voivodeship:  12
Start voivodeship
50%
End of voivodeship:  13
Start voivodeship
50%
End of voivodeship:  14
Start voivodeship
50%
End of voivodeship:  15
Start voivodeship
50%
End of voivodeship:  16
End webscripping!


## ***Data as DataFrame***

### Pandas library import

In [11]:
import pandas as pd

###  Changing the object to a DataFrame

In [12]:
df = pd.DataFrame(data)
pd.set_option('display.max_rows', None)

### Saving the object as a csv file

In [13]:
df.to_csv('houses_in_poland.csv', index=False)